In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def analyze_storage_data(csv_file_path):
    print("加载数据并计算基础统计量")
    try:
        df = pd.read_csv(csv_file_path)
        df['时间戳'] = pd.to_datetime(df['时间戳'])
        df['瞬时充电功率'] = df['充电功率(kW)']
        df['瞬时放电功率'] = df['放电功率(kW)']
        print("CSV文件加载成功！")
    except FileNotFoundError:
        print(f"错误：文件未找到在 {csv_file_path}。请确保文件存在且路径正确。")
        return None, None
    except KeyError as e:
        print(f"错误：CSV文件缺少必需的列或列名不匹配：{e}。请检查CSV文件中的列名。")
        print("期望的列名可能包含单位，例如 '充电功率(kW)' 和 '放电功率(kW)'。")
        return None, None

    numeric_columns = [
        '外部环境温度(°C)', '系统内部温度(°C)', '充电功率(kW)', '放电功率(kW)',
        '输入总能量(kWh)', '输出总能量(kWh)',
        '当前SOC(%)', '瞬时电压(V)', '瞬时电流(A)', '电网频率(Hz)', '电网电压(V)',
        '控制指令功率(kW)', '实际输出功率(kW)'
    ]

    statistics = {}
    for col in numeric_columns:
        if col in df.columns:
            stats = {
                '均值': df[col].mean(),
                '中位数': df[col].median(),
                '标准差': df[col].std(),
                '最小值': df[col].min(),
                '最大值': df[col].max()
            }
            statistics[col] = stats
            print(f"\n--- **{col}** 统计量 ---")
            for key, value in stats.items():
                print(f"{key}: {value:.2f}")
        else:
            print(f"警告：列 '{col}' 在CSV中不存在，跳过统计。")
            if col in ['充电功率(kW)', '放电功率(kW)', '输入总能量(kWh)', '输出总能量(kWh)', '当前SOC(%)', '实际输出功率(kW)']:
                 print(f"**警告：缺少关键列 '{col}'，这可能导致部分性能指标无法计算。**")

    return df, statistics

SYSTEM_DESIGN_CAPACITY_KWH = 2000
SYSTEM_MAX_POWER_KW = 500
SYSTEM_TOTAL_MASS_KG = 10000
SYSTEM_TOTAL_VOLUME_M3 = 20

def calculate_energy_capacity(df):
    if '输出总能量(kWh)' in df.columns and not df['输出总能量(kWh)'].empty:
        return df['输出总能量(kWh)'].max()
    print("警告：无法计算能量容量，缺少 '输出总能量(kWh)' 列或数据为空。")
    return np.nan

def calculate_power_rating():
    return SYSTEM_MAX_POWER_KW

def calculate_round_trip_efficiency(df):
    if '输入总能量(kWh)' in df.columns and '输出总能量(kWh)' in df.columns:
        total_input_energy = df['输入总能量(kWh)'].iloc[-1]
        total_output_energy = df['输出总能量(kWh)'].iloc[-1]
        if total_input_energy > 0:
            rte = (total_output_energy / total_input_energy) * 100
            return rte
        return 0.0
    print("警告：无法计算往返效率，缺少 '输入总能量(kWh)' 或 '输出总能量(kWh)' 列。")
    return np.nan

def calculate_energy_density(energy_capacity, total_mass, total_volume):
    if pd.isna(energy_capacity) or total_mass == 0 or total_volume == 0:
        return np.nan, np.nan
    energy_density_mass = energy_capacity / total_mass
    energy_density_volume = energy_capacity / total_volume
    return energy_density_mass, energy_density_volume

def calculate_power_density(max_power, total_mass, total_volume):
    if max_power == 0 or total_mass == 0 or total_volume == 0:
        return np.nan, np.nan
    power_density_mass = max_power / total_mass
    power_density_volume = max_power / total_volume
    return power_density_mass, power_density_volume

def calculate_power_throughput(cycle_energy_assumption_kwh=SYSTEM_DESIGN_CAPACITY_KWH, assumed_cycle_life=5000):
    return cycle_energy_assumption_kwh * assumed_cycle_life / 1000

def calculate_response_time(df):
    if '控制指令功率(kW)' not in df.columns or '实际输出功率(kW)' not in df.columns:
        return "无法计算 (缺少控制指令功率或实际输出功率列)"

    response_times = []
    df['指令变化'] = (df['控制指令功率(kW)'].diff().abs() > 0) & (df['控制指令功率(kW)'] != 0)
    
    command_change_indices = df[df['指令变化']].index

    for i in command_change_indices:
        command_time = df.loc[i, '时间戳']
        command_power = df.loc[i, '控制指令功率(kW)']
        
        for j in range(i + 1, min(i + 5, len(df))):
            actual_power = df.loc[j, '实际输出功率(kW)']
            
            if command_power > 0 and actual_power >= command_power * 0.9:
                response_time_delta = df.loc[j, '时间戳'] - command_time
                response_times.append(response_time_delta.total_seconds())
                break
            elif command_power < 0 and actual_power <= command_power * 0.9:
                response_time_delta = df.loc[j, '时间戳'] - command_time
                response_times.append(response_time_delta.total_seconds())
                break

    if response_times:
        return np.mean(response_times)
    return "无法计算 (无明显指令响应或数据粒度不足)"

def calculate_cycle_life(final_capacity_ratio=0.8, degradation_rate_per_cycle=0.00005):
    return (1 - final_capacity_ratio) / degradation_rate_per_cycle

def calculate_calendar_life(final_capacity_ratio=0.8, annual_degradation_rate=0.02):
    return (1 - final_capacity_ratio) / annual_degradation_rate

def calculate_ramp_rate(df):
    if '实际输出功率(kW)' not in df.columns:
        print("警告：无法计算爬坡率，缺少 '实际输出功率(kW)' 列。")
        return np.nan
    
    df['功率变化'] = df['实际输出功率(kW)'].diff().abs()
    df['时间间隔'] = df['时间戳'].diff().dt.total_seconds()
    ramp_rates = df[df['时间间隔'] > 0]['功率变化'] / df[df['时间间隔'] > 0]['时间间隔']
    if not ramp_rates.empty:
        return ramp_rates.max()
    return 0.0

def calculate_soc_range(df):
    if '当前SOC(%)' not in df.columns:
        print("警告：无法计算SOC范围，缺少 '当前SOC(%)' 列。")
        return np.nan
    min_soc = df['当前SOC(%)'].min()
    max_soc = df['当前SOC(%)'].max()
    return max_soc - min_soc

def calculate_self_discharge_rate(df):
    if '充电功率(kW)' not in df.columns or '放电功率(kW)' not in df.columns or '当前SOC(%)' not in df.columns:
        return "无法计算 (缺少充电功率、放电功率或当前SOC列)"

    df_idle = df[(df['充电功率(kW)'] == 0) & (df['放电功率(kW)'] == 0)]
    if len(df_idle) > 10:
        first_idle_row = df_idle.iloc[0]
        last_idle_row = df_idle.iloc[-1]
        soc_initial = first_idle_row['当前SOC(%)']
        soc_final = last_idle_row['当前SOC(%)']
        time_storage_hours = (last_idle_row['时间戳'] - first_idle_row['时间戳']).total_seconds() / 3600

        if time_storage_hours > 0 and soc_initial > 0:
            self_discharge = ((soc_initial - soc_final) / soc_initial) / time_storage_hours * 24
            return self_discharge
    return "无法计算 (静置数据不足)"

def analyze_temperature_characteristics(df):
    if '系统内部温度(°C)' not in df.columns:
        print("警告：无法分析温度特性，缺少 '系统内部温度(°C)' 列。")
        return {'平均内部温度': np.nan, '最低内部温度': np.nan, '最高内部温度': np.nan}
        
    avg_internal_temp = df['系统内部温度(°C)'].mean()
    min_internal_temp = df['系统内部温度(°C)'].min()
    max_internal_temp = df['系统内部温度(°C)'].max()
    return {
        '平均内部温度': avg_internal_temp,
        '最低内部温度': min_internal_temp,
        '最高内部温度': max_internal_temp
    }

def calculate_transient_overload_capability(df, rated_power=SYSTEM_MAX_POWER_KW):
    if '实际输出功率(kW)' not in df.columns:
        print("警告：无法计算瞬态过载能力，缺少 '实际输出功率(kW)' 列。")
        return np.nan, "缺少数据"

    overload_instances = df[df['实际输出功率(kW)'] > rated_power]
    if not overload_instances.empty:
        max_overload_power = overload_instances['实际输出功率(kW)'].max()
        overload_factor = max_overload_power / rated_power
        return overload_factor, "需要更精细的数据来计算持续时间"
    return 1.0, "无过载记录"

def calculate_c_rate(df, rated_capacity_kwh=SYSTEM_DESIGN_CAPACITY_KWH):
    if '充电功率(kW)' not in df.columns or '放电功率(kW)' not in df.columns:
        print("警告：无法计算C倍率，缺少 '充电功率(kW)' 或 '放电功率(kW)' 列。")
        return np.nan, np.nan
        
    avg_charge_power = df['充电功率(kW)'].mean()
    avg_discharge_power = df['放电功率(kW)'].mean()

    charge_c_rate = avg_charge_power / rated_capacity_kwh if rated_capacity_kwh > 0 else 0
    discharge_c_rate = avg_discharge_power / rated_capacity_kwh if rated_capacity_kwh > 0 else 0
    return charge_c_rate, discharge_c_rate

def calculate_esr(df):
    if '瞬时电压(V)' not in df.columns or '瞬时电流(A)' not in df.columns:
        return "无法计算 (缺少瞬时电压或瞬时电流列)"

    avg_voltage = df['瞬时电压(V)'].mean()
    avg_current = df['瞬时电流(A)'].mean()
    if avg_current != 0:
        return f"{abs(avg_voltage / avg_current):.4f} Ω"
    return "0.0000 Ω (平均电流为零)"

def analyze_thermal_degradation(df):
    if '系统内部温度(°C)' not in df.columns:
        return "无法评估 (缺少系统内部温度列)"
    return f"平均系统内部温度: {df['系统内部温度(°C)'].mean():.2f}°C (无法直接从短期数据计算衰减曲线)"

def calculate_insulation_resistance(df):
    return "大于 100 MΩ (假设符合标准，需要专用测量设备)"

def analyze_transient_response_characteristics(df):
    if '控制指令功率(kW)' not in df.columns or '实际输出功率(kW)' not in df.columns:
        return "无法分析 (缺少控制指令功率或实际输出功率列)"

    deviations = (df['控制指令功率(kW)'] - df['实际输出功率(kW)']).abs()
    avg_deviation = deviations.mean()
    max_deviation = deviations.max()
    return f"平均功率响应偏差: {avg_deviation:.2f} kW, 最大偏差: {max_deviation:.2f} kW (需要高频数据进行详细分析)"

def calculate_harmonic_content(df):
    return "无法计算 (需要高频电压/电流波形数据进行傅里叶变换)"

def calculate_frequency_support_capability(df):
    if '电网频率(Hz)' not in df.columns or '放电功率(kW)' not in df.columns or '充电功率(kW)' not in df.columns:
        return "无法评估 (缺少电网频率、充/放电功率列)"

    responsive_instances = df[
        ((df['电网频率(Hz)'] < 49.95) & (df['放电功率(kW)'] > 0)) |
        ((df['电网频率(Hz)'] > 50.05) & (df['充电功率(kW)'] > 0))
    ]
    if not responsive_instances.empty:
        return "具备频率响应能力 (需更精细数据评估响应速度和精度)"
    return "无明显频率响应记录"

def calculate_voltage_support_capability(df):
    return "无法计算 (需要无功功率数据，当前数据集中未包含)"

def analyze_ancillary_service_potential():
    return "取决于市场规则和系统综合性能 (功率、能量、响应时间、持续时间)"

if __name__ == '__main__':
    csv_file = 'data.csv'

    data_frame, basic_stats = analyze_storage_data(csv_file)

    if data_frame is not None and not data_frame.empty:
        print("\n" + "="*40)
        print("--- 2. 核心性能指标计算结果 ---")
        print("="*40)

        energy_capacity = calculate_energy_capacity(data_frame)
        print(f"**能量容量 (Energy Capacity):** {energy_capacity:.2f} kWh")

        power_rating = calculate_power_rating()
        print(f"**功率 (Power Rating):** {power_rating:.2f} kW")

        rte = calculate_round_trip_efficiency(data_frame)
        print(f"**往返效率 (Round-trip Efficiency, RTE):** {rte:.2f}%")

        energy_density_mass, energy_density_volume = calculate_energy_density(energy_capacity, SYSTEM_TOTAL_MASS_KG, SYSTEM_TOTAL_VOLUME_M3)
        print(f"**能量密度 (Energy Density):** {energy_density_mass:.2f} kWh/kg (按质量), {energy_density_volume:.2f} kWh/m³ (按体积)")

        power_density_mass, power_density_volume = calculate_power_density(power_rating, SYSTEM_TOTAL_MASS_KG, SYSTEM_TOTAL_VOLUME_M3)
        print(f"**功率密度 (Power Density):** {power_density_mass:.2f} kW/kg (按质量), {power_density_volume:.2f} kW/m³ (按体积)")

        power_throughput = calculate_power_throughput() 
        print(f"**功率吞吐量 (Power Throughput):** {power_throughput:.2f} MWh (基于假设)")

        response_time = calculate_response_time(data_frame)
        if isinstance(response_time, str):
            print(f"**响应时间 (Response Time):** {response_time}")
        else:
            print(f"**响应时间 (Response Time):** {response_time:.2f} 秒 (平均)")

        cycle_life = calculate_cycle_life()
        print(f"**循环寿命 (Cycle Life):** {cycle_life:.0f} 次 (基于衰减率假设)")

        calendar_life = calculate_calendar_life()
        print(f"**日历寿命 (Calendar Life):** {calendar_life:.1f} 年 (基于年衰减率假设)")

        ramp_rate = calculate_ramp_rate(data_frame)
        print(f"**爬坡率/斜坡率 (Ramp Rate):** {ramp_rate:.2f} kW/s")

        soc_range = calculate_soc_range(data_frame)
        print(f"**荷电状态范围 (SOC Range):** {soc_range:.2f}%")

        self_discharge_rate = calculate_self_discharge_rate(data_frame)
        print(f"**自放电率 (Self-Discharge Rate):** {self_discharge_rate} %/天")

        temp_chars = analyze_temperature_characteristics(data_frame)
        print(f"**温度特性 (Temperature Characteristics):** 平均内部温度 {temp_chars['平均内部温度']:.2f}°C, 范围 {temp_chars['最低内部温度']:.2f}-{temp_chars['最高内部温度']:.2f}°C")

        overload_factor, overload_duration_note = calculate_transient_overload_capability(data_frame)
        print(f"**瞬态过载能力 (Transient Overload Capability):** {overload_factor:.2f} 倍额定功率 ({overload_duration_note})")

        charge_c_rate, discharge_c_rate = calculate_c_rate(data_frame)
        print(f"**充放电C倍率 (C-Rate):** 充电 {charge_c_rate:.2f} C, 放电 {discharge_c_rate:.2f} C")

        esr = calculate_esr(data_frame)
        print(f"**等效串联电阻 (ESR):** {esr} (粗略估算，需专业测试)")

        thermal_degradation_note = analyze_thermal_degradation(data_frame)
        print(f"**热衰减 (Thermal Degradation):** {thermal_degradation_note}")

        insulation_resistance = calculate_insulation_resistance(data_frame)
        print(f"**绝缘电阻 (Insulation Resistance):** {insulation_resistance}")

        transient_response_note = analyze_transient_response_characteristics(data_frame)
        print(f"**瞬态响应特性 (Transient Response Characteristics):** {transient_response_note}")

        harmonic_content_note = calculate_harmonic_content(data_frame)
        print(f"**谐波含量 (Harmonic Content):** {harmonic_content_note}")

        frequency_support_note = calculate_frequency_support_capability(data_frame)
        print(f"**频率支持能力 (Frequency Support Capability):** {frequency_support_note}")

        voltage_support_note = calculate_voltage_support_capability(data_frame)
        print(f"**电压支持能力 (Voltage Support Capability):** {voltage_support_note}")

        ancillary_service_note = analyze_ancillary_service_potential()
        print(f"**辅助服务潜力 (Ancillary Service Potential):** {ancillary_service_note}")

    elif data_frame is not None and data_frame.empty:
        print("\n**错误：加载的CSV文件为空，无法进行分析。**")
    else:
        print("\n**数据加载失败，无法进行后续分析。**")


加载数据并计算基础统计量
CSV文件加载成功！

--- **外部环境温度(°C)** 统计量 ---
均值: 33.95
中位数: 33.80
标准差: 3.40
最小值: 25.90
最大值: 39.90

--- **系统内部温度(°C)** 统计量 ---
均值: 42.08
中位数: 43.05
标准差: 4.50
最小值: 30.30
最大值: 48.00

--- **充电功率(kW)** 统计量 ---
均值: 31.66
中位数: 0.00
标准差: 111.53
最小值: 0.00
最大值: 499.00

--- **放电功率(kW)** 统计量 ---
均值: 30.54
中位数: 0.00
标准差: 95.01
最小值: 0.00
最大值: 457.00

--- **输入总能量(kWh)** 统计量 ---
均值: 3200.52
中位数: 3619.00
标准差: 1849.62
最小值: 0.00
最大值: 5763.00

--- **输出总能量(kWh)** 统计量 ---
均值: 3111.92
中位数: 3028.00
标准差: 1585.17
最小值: 147.00
最大值: 5558.00

--- **当前SOC(%)** 统计量 ---
均值: 52.97
中位数: 30.00
标准差: 29.32
最小值: 30.00
最大值: 95.00

--- **瞬时电压(V)** 统计量 ---
均值: 399.45
中位数: 399.00
标准差: 2.79
最小值: 395.00
最大值: 404.90

--- **瞬时电流(A)** 统计量 ---
均值: 0.90
中位数: 0.60
标准差: 38.75
最小值: -115.60
最大值: 125.10

--- **电网频率(Hz)** 统计量 ---
均值: 50.00
中位数: 50.00
标准差: 0.03
最小值: 49.95
最大值: 50.05

--- **电网电压(V)** 统计量 ---
均值: 219.94
中位数: 219.90
标准差: 1.14
最小值: 218.00
最大值: 221.90

--- **控制指令功率(kW)** 统计量 ---
均值: 1.13
中位数: 0.00
标准差: 153.00
最小值: -457.00
